# Masterboot image with Signature Provider

This notebook describes how to use a custom remote signing service for signing the masterboot image.


In [1]:
%run ../init_notebook.ipynb

import pprint
import os

pp = pprint.PrettyPrinter(indent=4)

plugins_dir = 'plugins/'

SASP_PLUGIN = os.path.join(plugins_dir, 'sasp.py')
WORKSPACE = "workspace/mbimg/" # change this to path to your workspace
DATA_DIR = "data_mbimg/" # change this to path to your workspace
VERBOSITY = "-v" # verbosity of commands, might be -v or -vv for debug or blank for no additional info

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


### Signature Provider Plugin

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, open this [notebook](signature_provider.ipynb) and follow the instructions there. Once you are done, come back and continue here.

### Config File Setup

The masterboot image configuration file must be updated in order to integrate the custom Signature Provider.

The signature provider configuration must meet following rules:
  - Configuration key
    - key names `sign_provider` or `signProvider` are allowed

  - Configuration value
    - format `"type=<sp_type>;<key1>=<value1>;<key2>=<value2>;..."`
    - the `sp_type` has to match the sp_type class attribute defined in the custom signature provider(`plugins/sasp.py`)
    - the remaining key-value pairs are passed to the `__init__` method of the concrete Signature Provider
    - e.g.: `"type=file;file_path=private_key.pem"` will instantiate `spsdk.crypto.PlainFileSP(file_path='private_key.pem')`

In [2]:
import yaml
import os
import shutil
from spsdk.utils.misc import load_configuration
# choose family for the MCU
FAMILY = "rt5xx"

%! nxpimage $VERBOSITY mbi get-templates -f $FAMILY -o $WORKSPACE

CONFIG_PATH = os.path.join(WORKSPACE, "rt5xx_ext_xip_signed.yaml")
# just for verification that the template was generated
assert os.path.exists(CONFIG_PATH)

# we can remove all optional setting as we don't need them for thsi example
filtered_config = []
with open(CONFIG_PATH, 'r') as file:
    for line in file:
        if '[Optional]' not in line:
            filtered_config.append(line)

with open(CONFIG_PATH, 'w') as file:
    for line in filtered_config:
        file.write(f"{line}")

# update the config so it uses our custom Signature Provider
config = load_configuration(CONFIG_PATH)

# either the 'mainCertPrivateKeyFile' or 'signProvider' setting can be used
del config['mainCertPrivateKeyFile']
config['signProvider'] = 'type=sasp;key_number=0'
config['rootCertificate0File'] = 'root_k0_signed_cert0_noca.der.cert'
with open(CONFIG_PATH, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

# copy additional files needed for masterboot image creation
files_needed = ['my_application.bin', 'root_k0_signed_cert0_noca.der.cert']
for file in files_needed:
    shutil.copyfile(os.path.join(DATA_DIR, file), os.path.join(WORKSPACE, file))

pp.pprint(f"All files are ready in folder '{WORKSPACE}'")

nxpimage -v mbi get-templates -f rt5xx -o workspace/mbimg/ 
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ext_xip_plain.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ext_xip_crc.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ext_xip_signed.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ram_plain.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ram_crc.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ram_signed.yaml template file.
Creating c:\spsdk\examples\jupyter_examples\rt5xx_signature_provider\workspace\mbimg\rt5xx_ram_encrypted.yaml template file.
"All files are ready in folder 'workspace/mbimg/'"


### Execution

At this point, we have everything we need to run nxpimage application using remote HSM for image signing.

In [3]:
%! nxpimage $VERBOSITY mbi export --plugin $SASP_PLUGIN $CONFIG_PATH

# check if the signed image exists
output_file = os.path.join(WORKSPACE, config['masterBootOutputFile'])
assert os.path.exists(output_file)

nxpimage -v mbi export --plugin plugins/sasp.py workspace/mbimg/rt5xx_ext_xip_signed.yaml 
INFO:spsdk.image.mbi_mixin:RKTH: db31d46c717711a8231cbc38b1de8a6e8657e1f733e04c2ee4b62fcea59149fa
Success. (Master Boot Image: c:/spsdk/examples/jupyter_examples/rt5xx_signature_provider/workspace/mbimg/my_mbi.bin created.)


### HSM teardown

Last step is to stop custom HSM. In order to do that, open again the [notebook](sahsm.ipynb) and stop the running jupyter notebook code cell.